In [529]:
# Imports
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [530]:
# Custom dataset to train an NN with simple data and target tensors.
class CustomTensorDataset(Dataset):
    def __init__(self, data_tensor, target_tensor):
        self.data = data_tensor
        self.target = target_tensor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.target[index]

In [531]:
# Example tensors for learning addition. 

# Data tensor is 100,000 samples of 3 floats.
# Target sensor is the sum of these integers.
train_data_tensor = torch.rand(100000, 5)
train_target_tensor = torch.tensor([x[0] + x[1]**1 + x[2]**2 + x[3]**3 + x[4]**4 for x in train_data_tensor])

# Create dataset instance
train_dataset = CustomTensorDataset(train_data_tensor, train_target_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# And we do the same to create a separate test dataloader.
test_data_tensor = torch.rand(100000, 5)
test_target_tensor = torch.tensor([x[0] + x[1]**1 + x[2]**2 + x[3]**3 + x[4]**4 for x in test_data_tensor])

test_dataset = CustomTensorDataset(test_data_tensor, test_target_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [532]:
# Get training device.
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [533]:
# Define our NN structure to learn addition of 10 integers.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 5),
            nn.Sigmoid(),
            nn.Linear(5, 1),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [534]:
# Construct and move NN to device.
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=5, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)


In [ ]:
# Training params and loss function.
learning_rate = 1e-3
batch_size = 64
epochs = 100

# Since we're estimating a function (sum of all elements in input vector), mean squared error seems best.
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [536]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: Avg loss: {test_loss:>8f} \n")

In [537]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.631748  [   64/100000]
loss: 0.437422  [ 6464/100000]
loss: 0.431664  [12864/100000]
loss: 0.440256  [19264/100000]
loss: 0.569245  [25664/100000]
loss: 0.345800  [32064/100000]
loss: 0.519691  [38464/100000]
loss: 0.395577  [44864/100000]
loss: 0.342080  [51264/100000]
loss: 0.351146  [57664/100000]
loss: 0.444156  [64064/100000]
loss: 0.466659  [70464/100000]
loss: 0.452635  [76864/100000]
loss: 0.464691  [83264/100000]
loss: 0.411297  [89664/100000]
loss: 0.429613  [96064/100000]
Test Error: Avg loss: 0.410180 

Epoch 2
-------------------------------
loss: 0.360603  [   64/100000]
loss: 0.372220  [ 6464/100000]
loss: 0.342165  [12864/100000]
loss: 0.451489  [19264/100000]
loss: 0.517536  [25664/100000]
loss: 0.276158  [32064/100000]
loss: 0.421690  [38464/100000]
loss: 0.434635  [44864/100000]
loss: 0.253382  [51264/100000]
loss: 0.498888  [57664/100000]
loss: 0.435695  [64064/100000]
loss: 0.478404  [70464/100000]
loss: 0.491132  [76

KeyboardInterrupt: 

In [ ]:
# Try one more.
x = torch.rand(1,5)[0]
print(x)
print(x[0] + x[1]**1 + x[2]**2 + x[3]**3 + x[4]**4)

print(model(x))